<a href="https://colab.research.google.com/github/reservamos/xml-reader/blob/main/Lector_de_XMLs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
#!pip install pandas
from google.colab import drive
from google.colab import files
from IPython.display import display
from datetime import datetime
import pandas as pd
import os
import xml.etree.ElementTree as ET

drive.mount('/content/drive')

#@markdown Nombre de la carpeta a leer en Google /MyDrive.
folder = 'facturas' # @param {type:"string"}

folder_path = f'/content/drive/MyDrive/{folder}'

# Create a list to hold the data
data = []
total_sum = 0
taxes_sum = 0

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for xml_file in xml_files:
    file_path = os.path.join(folder_path, xml_file)
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract the Total attribute from the Comprobante element
    total = root.attrib.get('Total')  # Get the Total attribute with namespace
    total = float(total) if total else 0.0  # Convert to float or set to 0.0
    if total:
      total_sum += total  # Sum

    # Extract the FormaPago attribute from the Comprobante element
    payment_method = root.attrib.get('FormaPago')  # Get the FormaPago attribute with namespace

    # Extract the Nombre attribute from the Comprobante element
    creator = root.find('{http://www.sat.gob.mx/cfd/4}Emisor')  # Find the Emisor element
    name = creator.attrib.get('Nombre')  # Get the Nombre attribute with namespace

    # Extract the Taxes attribute from the Impuestos element directly under Comprobante
    taxes_element = root.find('{http://www.sat.gob.mx/cfd/4}Impuestos')  # Find the first Impuestos under Comprobante
    taxes = taxes_element.attrib.get('TotalImpuestosTrasladados') if taxes_element is not None else None
    taxes = float(taxes) if taxes else 0.0  # Convert to float or set to 0.0
    if taxes:
      taxes_sum += taxes  # Sum

    complement = root.find('{http://www.sat.gob.mx/cfd/4}Complemento')  # Find the UUID element
    if complement is not None:
        uuid_tag = complement.find('{http://www.sat.gob.mx/TimbreFiscalDigital}TimbreFiscalDigital')
        if uuid_tag is not None:
            uuid = uuid_tag.attrib.get('UUID')

    # Append the results to the data list
    data.append({
        'Total': total,
        'IVA': taxes,
        'Nombre': name,
        'Forma de Pago': payment_method,
        'Folio': uuid,
        'Archivo': xml_file
    })

# Create a DataFrame from the data
df = pd.DataFrame(data)

print("")
print(f"Suma Total: {total_sum}")
print(f"Suma IVA: {taxes_sum}")

df = df.sort_values(by='Folio')  # Sort by the UUID column
display(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Suma Total: 2724.0
Suma IVA: 373.61999999999995


,Total,IVA,Nombre,Forma de Pago,Folio,Archivo
0,1740.0,240.00,MARIA HORTENCIA STAUROFILA AZUELA FERNANDEZ,03,0a04999e-5dd4-4f98-b944-6de34502b5fb,AUFH620102I71FF1233.xml
2,384.0,52.96,FONDO NACIONAL DE INFRAESTRUCTURA,01,4094C408-4716-5BDB-9555-7B8AB7F32286,4094C408-4716-5BDB-9555-7B8AB7F32286.xml
1,600.0,80.66,SERVICIO LOMA DE LAGOS,04,b163dbf8-7609-4b72-86f1-c01e701e0ce4,20240809_SLL9810211B2_RES130904JH8_b163dbf8-76...
